In [ ]:
import pandas as pd

# Load tokenized messages
df = pd.read_csv("../data/dataset_telegram.csv")

#select 10 messages each from 3 different channelName
df_1 = df[df['Channel Title'] == 'Sheger online-store'].head(10)
df_2 = df[df['Channel Title'] == 'Leyueqa'].head(10)
df_3 = df[df['Channel Title'] == 'sinayelj'].head(10)

# Concatenate the selected messages
df = pd.concat([df_1, df_2, df_3])


In [10]:
# Generate a labelable CoNLL format with default label "O"
with open("conll_template.txt", "w", encoding="utf-8") as f:
    for message in df['Message'].dropna():
        tokens = str(message).split()
        for token in tokens:
            f.write(f"{token} O\n")
        f.write("\n")  # blank line between messages
print("✅ CoNLL format template generated for manual labeling.")

✅ CoNLL format template generated for manual labeling.
